In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from md04_func import *

## UNIT TEST

In [2]:
total=[]
filelist=[]
md04=parsingDoc(fileRead("6844_0481"))
PD=pd.DataFrame(md04["data"], columns=md04["header"])   
df=PD.iloc[:,0:7]    
total.append(pd.DataFrame(fehaParser(df), columns=header))

                           7,930.000  Exception occured
could not convert string to float: 
7930.000
['6844', '2018/05/29', 'Stock', '', 0, 0, 7930]
1121.000-
['6844', '2018/05/29', 'SafeSt', 'Safety Stock', 0, -9051, -1121]
1123.000-
['6844', '2017/10/19', 'Order', '5240194220', '000430', -2, -1123]
1129.000-
['6844', '2017/10/19', 'Order', '5240194220', '001840', -6, -1129]
1981.000-
['6844', '2018/05/14', 'IndReq', 'VSF', 0, -852, -1981]
7075.000-
['6844', '2018/05/21', 'IndReq', 'VSF', 0, -5094, -7075]
7076.000-
['6844', '2018/05/28', 'Order', '5240328013', '000040', -1, -7076]
7077.000-
['6844', '2018/05/28', 'Order', '5240341876', '000130', -1, -7077]
7080.000-
['6844', '2018/05/28', 'Order', '5240354663', '000080', -3, -7080]
7084.000-
['6844', '2018/05/28', 'Order', '5240355976', '000020', -4, -7084]
7085.000-
['6844', '2018/05/28', 'Order', '5240356137', '000110', -1, -7085]
7087.000-
['6844', '2018/05/28', 'Order', '5240359982', '000010', -2, -7087]
7090.000-
['6844', '2018/0

Partnr          6844
Date      2019/09/02
Type          PlOrd.
DocNum    0183191256
Pos             Stck
Change         21309
Avail          21313
Name: 222, dtype: object
Partnr          6844
Date      2019/09/02
Type          IndReq
DocNum           VSF
Pos                0
Change        -21309
Avail              4
Name: 223, dtype: object
Partnr          6844
Date      2019/10/04
Type          PlOrd.
DocNum    0183191257
Pos             Stck
Change         22078
Avail          22082
Name: 224, dtype: object
Partnr          6844
Date      2019/10/04
Type          IndReq
DocNum           VSF
Pos                0
Change        -22078
Avail              4
Name: 225, dtype: object
Partnr          6844
Date      2019/11/01
Type          PlOrd.
DocNum    0183191258
Pos             Stck
Change         21431
Avail          21435
Name: 226, dtype: object
Partnr          6844
Date      2019/11/01
Type          IndReq
DocNum           VSF
Pos                0
Change        -21431
Avail         

1285 -4
1281 -2
1279 -2
1277 -2
1275 -32
1243 -16
1227 -3
1224 -1
1223 -2
1221 -102
1119 -1
1118 -1
1117 -2
1115 -4
1111 -2
1109 -4
1105 -4
1101 -18
1083 -1
1082 -6
1076 -4
1072 -3
1069 -4
1065 -4
1061 -1
1060 -2
1058 -2
1056 -2
1054 -100
954 -10
944 -168
776 -16
760 -6
754 -9
745 -40
705 -10
695 -2
693 -1
692 -2
4 -2024
14 -2020
3757 -2006
1751 -300
1451 -300
1151 -1
1150 -1
1149 -1
1148 -80
1068 -100
968 -1
967 -10
12 -4043
2778 -4031
37 -1253
2517 -1216
1301 -300
2730 -1729
1001 -2
999 -4
995 -4
991 -10
981 -2
979 -6
973 -4
969 -2
21 -1763
6508 -1742
4766 -2
4764 -2
4762 -4
4758 -20
4738 -20
4718 -1
4717 -2
4715 -25
4690 -5
4685 -5
4680 -10
4670 -2730
1940 -2
1938 -2
1936 -6
1930 -2
1928 -9
1919 -68
1851 -4
1847 -4
1843 -2
1841 -2
1839 -1
1838 -3
68 -3165
5 -3097
3 -3092
5618 -3089
2529 -2
2527 -3
2524 -6
2518 -7
2511 -2
2509 -2
2507 -2
2505 -2
2503 -6
2497 -1
2496 -6
2490 -6
2484 -6
2478 -2
2476 -10
2466 -2
2464 -2
2462 -5
2457 -8
2449 -1
!!!!!!!!!!!!!!!!!!!!!!!!
[['6844', '2018/05

In [3]:
pd.DataFrame(total[0])

,Partnr,SO_Date,SO_MRP,SO_NR,SO_POS,SO_QTY,PO_DATE,PO_MRP,PO_NR,PO_POS,PO_QTY
0,6844,2018/05/29,SafeSt,Safety Stock,0,-7930,2018/05/29,Stock,,0,7930
1,6844,2018/05/29,SafeSt,Safety Stock,0,-120,2018/06/08,PlOrd.,0183191216,Stck,120
2,6844,2018/05/29,SafeSt,Safety Stock,0,-1001,2018/06/08,PlOrd.,0183191217,Stck,1001
3,6844,2017/10/19,Order,5240194220,000430,-2,2018/06/08,PlOrd.,0183191217,Stck,2
4,6844,2017/10/19,Order,5240194220,001840,-6,2018/06/08,PlOrd.,0183191217,Stck,6
5,6844,2018/05/28,Order,5240328013,000040,-1,2018/06/08,PlOrd.,0183191217,Stck,1
6,6844,2018/05/28,Order,5240341876,000130,-1,2018/06/08,PlOrd.,0183191217,Stck,1
7,6844,2018/05/28,Order,5240354663,000080,-3,2018/06/08,PlOrd.,0183191217,Stck,3
8,6844,2018/05/28,Order,5240355976,000020,-4,2018/06/08,PlOrd.,0183191217,Stck,4
9,6844,2018/05/28,Order,5240356137,000110,-1,2018/06/08,PlOrd.,0183191217,Stck,1


## ACTUAL RUNNING (for MD04list.txt)

In [ ]:
total=[]
filelist=[]
f=open("MD04list.txt","r", encoding="UTF-8")
list=f.readlines()
for i in list :      
    md04=parsingDoc(fileRead(i.replace("\n","")))    
    PD=pd.DataFrame(md04["data"], columns=md04["header"])   
    #PD.plot(x="Date", kind="bar", title=md04['meta'][0])
    df=PD.iloc[:,0:7]    
    total.append(pd.DataFrame(fehaParser(df), columns=header))

## EXCEL FILE 

In [4]:
final=pd.DataFrame()
for i in total : 
        final=final.append(i)

In [15]:
#To save excel file 
writer=pd.ExcelWriter(str(datetime.datetime.now().date()) + 'output.xlsx')
final.to_excel(writer,'Sheet1')
writer.save()